In [24]:
from datasets import load_dataset
import random
import pandas as pd
import duckdb
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa
import os
import polars as pl
import ast

GDSC data lownloaded from: https://www.cancerrxgene.org/downloads/bulk_download

In [2]:
df1 = pd.read_excel('/hpc/home/yc583/Tahoe100M_practice/data/GDSC1_fitted_dose_response_27Oct23.xlsx')
df2 = pd.read_excel('/hpc/home/yc583/Tahoe100M_practice/data/GDSC2_fitted_dose_response_27Oct23.xlsx')
gdsc = pd.concat([df1, df2], ignore_index=True)
gdsc

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC1,342,15580432,684057,ES5,SIDM00263,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,3.966813,0.985678,0.026081,1.299144
1,GDSC1,342,15580806,684059,ES7,SIDM00269,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.692090,0.972690,0.110059,0.156076
2,GDSC1,342,15581198,684062,EW-11,SIDM00203,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.477990,0.944459,0.087019,-0.035912
3,GDSC1,342,15581542,684072,SK-ES-1,SIDM01111,UNCLASSIFIED,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.033564,0.950758,0.016290,-0.434437
4,GDSC1,342,15581930,687448,COLO-829,SIDM00909,SKCM,1,Erlotinib,EGFR,EGFR signaling,1045,Y,0.007813,2.0,2.966007,0.954778,0.180255,0.401702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575192,GDSC2,343,16188242,1659928,SNU-175,SIDM00216,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.127082,0.976746,0.074498,0.156872
575193,GDSC2,343,16188695,1660034,SNU-407,SIDM00214,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,8.576377,0.913378,0.057821,-1.626959
575194,GDSC2,343,16188953,1660035,SNU-61,SIDM00194,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.519636,0.975001,0.058090,0.608442
575195,GDSC2,343,16189493,1674021,SNU-C5,SIDM00498,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.694579,0.969969,0.101013,0.809684


In [3]:
# check duplicated drug_name - cell_line pairs within and between 2 versions of data
len(duckdb.query("SELECT DRUG_NAME, CELL_LINE_NAME FROM gdsc GROUP BY DRUG_NAME, CELL_LINE_NAME HAVING COUNT(*)>1").df())

99012

In [4]:
# check duplicated drug_name - cell_line pairs within single version of data
len(duckdb.query("SELECT DRUG_NAME, CELL_LINE_NAME, DATASET FROM gdsc GROUP BY DRUG_NAME, CELL_LINE_NAME, DATASET HAVING COUNT(*)>1").df())

24355

For duplicated pairs between 2 versions of release, use the IC50 in GDSC2. For duplicated pairs within the same version, use the aggregated value of IC50.

In [5]:
def exp_mean(x):
    return np.log(np.exp(x).mean())
    
gdsc['LN_IC50'] = gdsc.groupby(['DRUG_NAME', 'CELL_LINE_NAME', 'DATASET'], as_index=False)['LN_IC50'].transform(exp_mean)
gdsc = gdsc.drop_duplicates(subset=['DRUG_NAME', 'CELL_LINE_NAME'], keep='last')
gdsc = gdsc[['DRUG_NAME', 'CELL_LINE_NAME', 'LN_IC50']]

In [ ]:
# get mapping table between cell-line name and cell-line cellosaur id
cell_line_metadata = load_dataset(
    "vevotx/Tahoe-100M",
    name="cell_line_metadata",
    split="train").to_pandas()

cell_line_mapping = cell_line_metadata.drop_duplicates(subset=['cell_name', 'Cell_ID_Cellosaur'])
cell_line_mapping = cell_line_mapping[['cell_name', 'Cell_ID_Cellosaur']]

In [6]:
gdsc.to_csv('/hpc/home/yc583/Tahoe100M_practice/data/GDSC_processed.csv', index=False)

Down-sample Tahoe data to 10M. Saved the data to parquet database.

In [ ]:
output_dir = '/hpc/group/biostat/yc583/Tahoe_downsampled_parquet'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
chunks = pd.read_csv('/hpc/group/naderilab/eleanor/prose_data/data/Tahoe_downsampled.txt.gz', 
                     sep='\t',
                     compression='gzip',
                     chunksize=100, 
                     low_memory=False)
for chunk in chunks:
    table = pa.Table.from_pandas(chunk)
    pq.write_to_dataset(
        table, 
        root_path=output_dir, 
        compression="snappy" ,
        # existing_data_behavior="delete_matching" 
    )
        

Merge down-sampled Tahoe data with processed GDSC data

In [2]:
output_dir = '/hpc/group/biostat/yc583/Tahoe_downsampled_parquet'
tahoe = duckdb.read_parquet(f"{output_dir}/**/*.parquet", hive_partitioning=True)
gdsc = pl.scan_csv('/hpc/home/yc583/Tahoe100M_practice/data/GDSC_processed.csv')

In [3]:
# get mapping table between cell-line name and cell-line cellosaur id
cell_line_metadata = load_dataset(
    "vevotx/Tahoe-100M",
    name="cell_line_metadata",
    split="train").to_pandas()

cell_line_mapping = cell_line_metadata.drop_duplicates(subset=['cell_name', 'Cell_ID_Cellosaur'])
cell_line_mapping = cell_line_mapping[['cell_name', 'Cell_ID_Cellosaur']]
cell_line_mapping.to_csv('/hpc/home/yc583/Tahoe100M_practice/data/cell_line_mapping_table.csv', index=False)


Resolving data files:   0%|          | 0/3388 [00:00<?, ?it/s]

In [4]:
cell_line = pl.scan_csv('/hpc/home/yc583/Tahoe100M_practice/data/cell_line_mapping_table.csv')
tahoe_cell = duckdb.sql(
    """
    SELECT * 
    FROM tahoe t JOIN cell_line c ON t.cell_line_id = c.Cell_ID_Cellosaur
    """
)

In [5]:
tahoe_gdsc = duckdb.sql(
    """
    SELECT *
    FROM tahoe_cell tc JOIN gdsc g ON tc.drug = g.DRUG_NAME AND tc.cell_name = g.CELL_LINE_NAME
    """
).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
tahoe_gdsc.to_csv('/hpc/home/yc583/Tahoe100M_practice/data/Tahoe_GDSC_raw.csv', index=False)

In [42]:
tahoe_gdsc = pd.read_csv('/hpc/home/yc583/Tahoe100M_practice/data/Tahoe_GDSC_raw.csv')
tahoe_gdsc = tahoe_gdsc[['genes','expressions','canonical_smiles','LN_IC50']]
tahoe_gdsc_copy = tahoe_gdsc.copy()
tahoe_gdsc_copy['genes'] = tahoe_gdsc_copy['genes'].apply(lambda x: list(map(int, (x.split(','))[1:])))
tahoe_gdsc_copy['expressions'] = tahoe_gdsc_copy['expressions'].apply(lambda x: list(map(float, (x.split(','))[1:])))

tahoe_gdsc_copy

,genes,expressions,canonical_smiles,LN_IC50
0,"[12, 19, 20, 21, 32, 35, 43, 55, 56, 70, 78, 7...","[1.0, 1.0, 2.0, 1.0, 3.0, 1.0, 4.0, 1.0, 2.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-1.544005
1,"[14, 19, 24, 26, 33, 40, 45, 55, 59, 69, 73, 7...","[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,0.121983
2,"[5, 56, 78, 79, 139, 140, 153, 167, 174, 195, ...","[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,0.121983
3,"[5, 19, 43, 70, 106, 112, 113, 114, 137, 231, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-1.544005
4,"[3, 21, 45, 70, 95, 103, 106, 114, 138, 139, 1...","[1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,1.171050
...,...,...,...,...
127731,"[22, 45, 78, 95, 103, 128, 157, 187, 233, 234,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-0.768822
127732,"[10, 19, 26, 32, 42, 43, 68, 70, 85, 95, 112, ...","[1.0, 1.0, 1.0, 1.0, 2.0, 6.0, 2.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-3.075120
127733,"[11, 14, 32, 44, 52, 77, 134, 141, 156, 221, 2...","[1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-0.768822
127734,"[11, 19, 26, 41, 42, 54, 56, 68, 82, 84, 94, 9...","[2.0, 1.0, 3.0, 1.0, 2.0, 1.0, 4.0, 1.0, 1.0, ...",CC1CCC2CC(C(=CC=CC=CC(CC(C(=O)C(C(C(=CC(C(=O)C...,-3.075120


In [43]:
tahoe_gdsc_copy.to_csv('/hpc/home/yc583/Tahoe100M_practice/data/Tahoe_GDSC_processed.csv', index=False)